In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
test = pd.read_csv('test.csv') 
train = pd.read_csv('train.csv') 
val = pd.read_csv('val.csv') 

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

In [ ]:
print(val.shape)
val.head()

In [ ]:
train.hist(figsize=(16,12))
plt.show()

In [ ]:
test.hist(figsize=(16,12))
plt.show()

In [ ]:
val.hist(figsize=(16,12))
plt.show()

In [ ]:
print(train.info())
print(test.info())
print(val.info())

W danych nie ma braków, zbiór test ma dodatkową kolumnę class informującą czy wiersz jest obserwacją odstającą, po rozkładach widać prawdopodobne miejsca występowania outllierów w zbiorach val i test oraz ich brak w train.

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import f1_score, precision_score, recall_score

GMM zwraca prawdopodobieństwo przyporządkowania obserwacji do klastra, więc obserwacje o niskim i odstającym od reszty prawdopodobieństwie mogą być traktowane jako outliery.

Za pomocą funcji bic wyznaczmy odpowiednią liczbę klastrów - im mniejsza warość funkcji tym lepiej.

In [ ]:
bic = []
for i in range(1, 13):
    gmm = GaussianMixture(n_components = i, random_state = 29, covariance_type = "full")
    gmm.fit(train)
    bic.append(gmm.bic(train))

In [ ]:
sns.lineplot(data = bic)

Najmniejszą wartość funkcja bic osiąga dla 1 klastra, stwórzmy więc właściwy model:

In [ ]:
gmm = GaussianMixture(covariance_type='full', n_components=1, random_state=29)
gmm.fit(train)

W zbiorze train nie ma outlierów a w val są, porównajmy więc prawdopodbieństwa i wyznaczmy na ich podstawie treshold.

In [ ]:
train_prop = gmm.score_samples(train)
val_prop = gmm.score_samples(val)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (14, 5))

sns.lineplot(data = train_prop, ax = ax1)
ax1.set_title('Propabilities on train data')

sns.lineplot(data = val_prop, ax = ax2)
ax2.set_title('Propabilities on validation data')

plt.show()

In [ ]:
#bierzemy treshold -30
t = -32

In [ ]:
test_prop = gmm.score_samples(test.drop('class', axis = 1))

In [ ]:
test_pred = np.where(test_prop < t, 1, 0)

In [ ]:
print(f'F1: {f1_score(test["class"], test_pred)}\n' + 
      f'Precision: {precision_score(test["class"], test_pred)}\n' + 
      f'Recall: {recall_score(test["class"], test_pred)}')

Metryki osiągają wysokie wyniki, zatem algorytm GMM nadaje się do wyszukiwania outlierów.